<a href="https://colab.research.google.com/github/Rashneema/C114-V2-text-data-tokenization/blob/main/PRO_C114_Text_data_tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load the Dataset

In [ ]:
!git clone https://github.com/procodingclass/PRO-C114-Text-Sentiment-Dataset

Cloning into 'PRO-C114-Text-Sentiment-Dataset'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 20 (delta 2), reused 0 (delta 0), pack-reused 11
Unpacking objects: 100% (20/20), 2.94 MiB | 5.00 MiB/s, done.


## Pandas:
**Pandas** is an open-source library built on top of Numpy and Matplotlib. It provides high-performance, easy-to-use data structures and data analysis tools.

A DataFrame is a two-dimensional data structure, i.e., data is aligned in a tabular fashion in rows and columns. Pandas DataFrame consists of three principal components, the data, rows, and columns.

Pandas DataFrame will be created by loading the datasets from existing MS Excel files, CSV files or SQL Database. Pandas DataFrame can also be created from the lists, dictionaries etc.



In [ ]:
#Import pandas
import pandas as pd
#read excel file
df = pd.read_excel("/content/PRO-C114-Text-Sentiment-Dataset/text-emotion-training-dataset.xlsx")
#display first five entries of training dataset
df.head()

,Text_Emotion
0,i didnt feel humiliated;sadness
1,i can go from feeling so hopeless to so damned...
2,im grabbing a minute to post i feel greedy wro...
3,i am ever feeling nostalgic about the fireplac...
4,i am feeling grouchy;anger


## Split the rows in two columns as Text and Emotions



In [ ]:
#Use split() method to separate the Text and Emotions
train_data = pd.DataFrame(df["Text_Emotion"].str.split(";",1).tolist(),columns=["Text","Emotion"])

## Giving labels to Emotions

In [ ]:
train_data.head()

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [ ]:
#Find unique emotions
train_data["Emotion"].unique()

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'],
      dtype=object)

In [ ]:
#Create a Dictionary to replace emotions with labels
encode_emotions={"anger": 0, "fear": 1, "joy": 2, "love": 3, "sadness": 4, "surprise": 5}

In [ ]:
#Let's replace the emotion with the respective labels. To serve this purpose we will use the replace() method.
train_data.replace(encode_emotions, inplace = True)
train_data.head()

,Text,Emotion
0,i didnt feel humiliated,4
1,i can go from feeling so hopeless to so damned...,4
2,im grabbing a minute to post i feel greedy wrong,0
3,i am ever feeling nostalgic about the fireplac...,3
4,i am feeling grouchy,0


## Convert Dataframe to list of dataset

Machine learning model takes numbers as input. This
means that we will need to convert the text into numerical
value. 

In [ ]:
# Convert Dataframe to list of dataset

training_sentences = []
training_labels = []

# append text and emotions in the list using the 'loc' method

for i in range(len(train_data)):
  
  sentence = train_data.loc[i, "Text"]
  training_sentences.append(sentence)

  label = train_data.loc[i, "Emotion"]
  training_labels.append(label)

In [ ]:
#Check a random text and label of the list

training_sentences[50], training_labels[50]

('i need to feel the dough to make sure its just perfect', 2)

## Tokenization & Padding
The act of converting text into numbers is known as **Tokenization**. The Tokenizer class of Keras is used for encoding text input into integer sequence.

**Padding** is important to make all the sentences contain the same number of words. Zero is used for padding the tokenized sequence to make text contain the same number of tokens.

# New Section

In [ ]:
#import Tokenizer from tensorflow

import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer

#Define parameters for Tokenizer

vocab_size = 10000
embedding_dim = 16
oov_tok = "<OOV>"
training_size = 20000

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

#Create a word_index dictionary

word_index = tokenizer.word_index

#Check numeral value assigned to a word

word_index["the"]

6

In [ ]:
len(word_index)

15213

In [ ]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)

print(training_sequences[0])
print(training_sequences[1])
print(training_sequences[2])

[2, 139, 3, 679]
[2, 40, 101, 60, 8, 15, 494, 5, 15, 3496, 553, 32, 60, 61, 128, 148, 76, 1480, 4, 22, 1255]
[17, 3060, 7, 1149, 5, 286, 2, 3, 495, 438]


In [ ]:
train_data.head(3)

,Text,Emotion
0,i didnt feel humiliated,4
1,i can go from feeling so hopeless to so damned...,4
2,im grabbing a minute to post i feel greedy wrong,0


In [ ]:
word_index["grabbing"]

3060

**Padding** It is important to make all the sentences contain the same number of words. Zero is used for padding the tokenized sequence to make text contain the same number of tokens. 

In [ ]:
#Define parameters for pad_sequences

from tensorflow.keras.preprocessing.sequence import pad_sequences

padding_type='post'
max_length = 100
trunc_type='post'


training_padded = pad_sequences(training_sequences, maxlen=max_length, 
                                padding=padding_type, truncating=trunc_type)

training_padded

array([[   2,  139,    3, ...,    0,    0,    0],
       [   2,   40,  101, ...,    0,    0,    0],
       [  17, 3060,    7, ...,    0,    0,    0],
       ...,
       [   2,    3,  327, ...,    0,    0,    0],
       [   2,    3,   14, ...,    0,    0,    0],
       [   2,   47,    7, ...,    0,    0,    0]], dtype=int32)